In [25]:
import advertools as a
import pandas as pd
import bs4
import requests

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [26]:
# Create a basic list and dataframe for holding the scraped web data
urls = list()
df = pd.DataFrame(columns=['url', 'Name', 'Difficulty', 'Distance', 'Type', 'High Elev', 'Low Elev', 'Elev Gained', 
                           'Elev Lost', 'Avg Grade', 'Max Grade'])

In [27]:
temp_df = pd.DataFrame()

for i in range(0, 100):
    site_url = 'https://www.hikingproject.com/sitemap' + str(i) + '.xml.gz'
    temp_df = a.sitemap_to_df(site_url)
    temp_df = temp_df[temp_df["loc"].str.contains('\/trail\/')]["loc"]

    # only add the url to the list, not the index number from the dataframe
    for label, content in temp_df.items():
        urls.append(content)

2021-07-25 03:03:46,320 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.hikingproject.com/sitemap0.xml.gz
2021-07-25 03:03:46,471 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.hikingproject.com/sitemap1.xml.gz
2021-07-25 03:03:46,619 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.hikingproject.com/sitemap2.xml.gz
2021-07-25 03:03:46,804 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.hikingproject.com/sitemap3.xml.gz
2021-07-25 03:03:46,960 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.hikingproject.com/sitemap4.xml.gz
2021-07-25 03:03:47,116 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.hikingproject.com/sitemap5.xml.gz
2021-07-25 03:03:47,295 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.hikingproject.com/sitemap6.xml.gz
2021-07-25 03:03:47,443 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.hikingproject.com/sitemap7.xml.gz
2021-07-25 03:03:47,592 | INFO |

2021-07-25 03:03:57,369 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.hikingproject.com/sitemap68.xml.gz
2021-07-25 03:03:57,520 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.hikingproject.com/sitemap69.xml.gz
2021-07-25 03:03:57,670 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.hikingproject.com/sitemap70.xml.gz
2021-07-25 03:03:57,852 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.hikingproject.com/sitemap71.xml.gz
2021-07-25 03:03:58,001 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.hikingproject.com/sitemap72.xml.gz
2021-07-25 03:03:58,155 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.hikingproject.com/sitemap73.xml.gz
2021-07-25 03:03:58,333 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.hikingproject.com/sitemap74.xml.gz
2021-07-25 03:03:58,485 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.hikingproject.com/sitemap75.xml.gz
2021-07-25 03:03:58,635 

In [28]:
# Display urls
urls, len(urls)

(['https://www.hikingproject.com/trail/7000000/mount-sanitas-loop',
  'https://www.hikingproject.com/trail/7000100/lower-big-bluestem-trail',
  'https://www.hikingproject.com/trail/7000500/milner-pass-fall-river-pass-trail',
  'https://www.hikingproject.com/trail/7000600/boulder-ridge-trail',
  'https://www.hikingproject.com/trail/7001600/middle-fork-taylor-creek-trail',
  'https://www.hikingproject.com/trail/7001800/fir-trail',
  'https://www.hikingproject.com/trail/7001900/griff-creek-trail',
  'https://www.hikingproject.com/trail/7002200/south-creek-trail',
  'https://www.hikingproject.com/trail/7002400/shingletown-loop',
  'https://www.hikingproject.com/trail/7002500/red-trail',
  'https://www.hikingproject.com/trail/7002600/carroll-canyon-trail',
  'https://www.hikingproject.com/trail/7002800/modoc-line-rail-trail-viewland-segment',
  'https://www.hikingproject.com/trail/7003000/beanos-hill-climb',
  'https://www.hikingproject.com/trail/7003300/unkar-delta',
  'https://www.hikingp

In [29]:
def scrape_page(site_url):
    """ Scrapes a given page for the desired fields.

    Args:
        site_url: the FQDN of the url to scrape

    Returns:
        d: a dictionary object that includes the following trail stats: site_url, trail_name, stars, difficulty, dist,
        trail_type, elev_high, elev_low, elev_gain, elev_lost, avg_grade, max_grade, dogs, features, gps_cords
    """
    page = requests.get(site_url)
    soup = bs4.BeautifulSoup(page.content, 'html.parser')

    # checks to make sure page exists
    if page.status_code == 200:
        trail_name = soup.find(id='trail-title').text.strip()
        difficulty = soup.find(class_='difficulty-text').text.strip()

        stats = soup.find(id='trail-stats-bar')
        stat_block = stats.find('div', class_='stat-block ml-2 mr-1 mt-1').text.strip().split('\n\n\n')
        dist = stat_block[0]
        trail_type = stat_block[2]
        stat_block2 = stats.find('div', class_='stat-block mx-1 mt-1').text.strip().split('\n\n')
        elev_high = (stat_block2[0]+stat_block2[2]).strip()
        elev_low = (stat_block2[3]+stat_block2[5]).strip()
        stat_block3 = stats.find('div', class_='stat-block mx-1 mt-1').nextSibling.nextSibling.text.strip().split('\n\n')
        elev_gain = (stat_block3[0]+stat_block2[2]).strip()
        elev_lost = (stat_block3[3]+stat_block2[5]).strip()
        stat_block4 = stats.find('div', class_='stat-block ml-1 mt-1').text.strip().split('\n')
        avg_grade = stat_block4[1].strip().replace("(", "").replace(")", "")
        max_grade = stat_block4[4].strip().replace("(", "").replace(")", "")

        # create dictionary with trail stats and return
        trail_stats = {'url': site_url, 'Name': trail_name, 'Difficulty': difficulty, 'Distance': dist, 'Type': trail_type, 
                       'High Elev': elev_high, 'Low Elev': elev_low, 'Elev Gained': elev_gain, 'Elev Lost': elev_lost, 
                       'Avg Grade': avg_grade, 'Max Grade': max_grade}
        return trail_stats
    else:
        return None

['https://www.hikingproject.com/trail/7000000/mount-sanitas-loop',
 'https://www.hikingproject.com/trail/7000100/lower-big-bluestem-trail',
 'https://www.hikingproject.com/trail/7000500/milner-pass-fall-river-pass-trail',
 'https://www.hikingproject.com/trail/7000600/boulder-ridge-trail',
 'https://www.hikingproject.com/trail/7001600/middle-fork-taylor-creek-trail']

In [30]:
for url in urls:
    d = scrape_page(url)
    if d is not None:
        df = df.append(d, ignore_index=True)

In [31]:
df

,url,Name,Difficulty,Distance,Type,High Elev,Low Elev,Elev Gained,Elev Lost,Avg Grade,Max Grade
0,https://www.hikingproject.com/trail/7000000/mo...,Mount Sanitas Loop,Intermediate/Difficult,3.2 Miles,Loop,"6,780' High","5,521' Low","1,281' High","1,280' Low",9°,23°
1,https://www.hikingproject.com/trail/7000100/lo...,Lower Big Bluestem Trail,Easy/Intermediate,2.0 Miles,Point to Point,"5,877' High","5,504' Low",397' High,25' Low,2°,9°
2,https://www.hikingproject.com/trail/7000500/mi...,Milner Pass - Fall River Pass Trail,Intermediate,3.5 Miles,Point to Point,"11,764' High","11,092' Low",13' High,685' Low,2°,4°
3,https://www.hikingproject.com/trail/7000600/bo...,Boulder Ridge Trail,Intermediate,1.2 Miles,Point to Point,"6,815' High","6,631' Low",217' High,221' Low,4°,10°
4,https://www.hikingproject.com/trail/7001600/mi...,Middle Fork Taylor Creek Trail,Intermediate,4.6 Miles,Out and Back,"5,921' High","5,482' Low",455' High,455' Low,2°,4°
...,...,...,...,...,...,...,...,...,...,...,...
59458,https://www.hikingproject.com/trail/7101599/tw...,Twig Trail,Intermediate,1.5 Miles,Point to Point,"1,720' High","1,404' Low",349' High,89' Low,3°,5°
59459,https://www.hikingproject.com/trail/7101799/sa...,Saint Elmo Peak,Intermediate/Difficult,0.9 Miles,Point to Point,"6,391' High","5,263' Low","1,131' High",7' Low,13°,21°
59460,https://www.hikingproject.com/trail/7101899/sk...,Skyline Drive Trail,Intermediate/Difficult,1.4 Miles,Point to Point,"5,504' High","5,007' Low",15' High,512' Low,4°,11°
59461,https://www.hikingproject.com/trail/7101999/be...,Bee Tree Trail,Easy,2.2 Miles,Out and Back,13' High,10' Low,14' High,13' Low,0°,1°


In [33]:
df.to_csv('data/raw_trail_data.csv', index=False)